In [4]:
import numpy as np
import pandas as pd

In [9]:
class Die():
    '''
    Docstring
    '''
    def __init__(self, face_symbols):
        if isinstance(face_symbols, np.ndarray):
            if face_symbols.dtype.char == U:
                
        else:
            raise TypeError('Face Argument must be NumPy array')
            
    
    def initializer():
        pass
    
    def change_side_weight():
        pass
    
    def roll():
        pass
    
    def current_state():
        pass

In [18]:
i = np.array(['hhhhhhh', 'gggggg'])
i.dtype

dtype('<U7')